In [51]:
import serial as ser
from serial.tools import list_ports
from crcmod import mkCrcFun
import time

In [34]:

# Modbus CRC16校验函数（多项式0x8005，初始值0xFFFF）
crc16 = mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)


In [35]:
list_ports = list_ports.comports()

In [36]:
for port in list_ports:
    print(port)

/dev/ttyUSB0 - FT232R USB UART - FT232R USB UART


In [37]:
def list_com_ports():
    """获取系统可用串口列表"""
    
    ports = ser.tools.list_ports.comports()
    return [port.device for port in ports]

In [38]:
list_com_ports()

['/dev/ttyUSB0']

In [39]:
ser_port = ser.Serial(
        port='/dev/ttyUSB0',
        baudrate=115200,
        bytesize=ser.EIGHTBITS,
        parity=ser.PARITY_NONE,
        stopbits=ser.STOPBITS_ONE,
        timeout=1
    )

In [40]:
HEADER = 0x0706
ADDR = 0x0001
DATALEN = 0x04
CRC16 = 0x0000

In [41]:
buf = []

In [46]:
def gen_cmd(addr ,data, length):
    """生成Modbus命令帧"""
    
    # 构造基础数据包
    packet = bytearray()
    packet.extend(HEADER.to_bytes(2, 'big'))  # 大端序包头
    packet.extend(ADDR.to_bytes(2, 'little'))    # 设备地址
    
    # 添加数据部分
    packet.extend(length.to_bytes(1, 'big'))  # 数据长度
    packet.extend(data.to_bytes(length, 'little'))                     # 数据本体
    
    # 计算并附加CRC
    crc = crc16(packet)
    packet.extend(crc.to_bytes(2, 'big'))
    
    print(packet.hex())
    return bytes(packet)

    

In [48]:
buf = gen_cmd(0x0001,8000,4)

0706010004401f0000ec69


In [64]:
# 发送完整数据包
ser_port.write(buf)
ser_port.flush()  # 确保数据完全发送

# # 读取响应（根据协议设置合适的读取超时和字节数）
# response = ser_port.read(size=32)  # 根据实际响应长度调整

# # 打印通信结果
print(f">: {buf.hex(' ')}")
# print(f"收到响应: {response.hex(' ') if response else '无响应'}")


>: 07 06 01 00 04 40 1f 00 00 ec 69


In [65]:
timeout = 0.5        # 单次读取超时时间
buffer = bytearray()

while ser_port.is_open:
    chunk = ser_port.read(1)  # 每次读取1字节
    if chunk:
        buffer.extend(chunk)
    else:
        print("<:", buffer.hex(' ') if buffer else "NO DATA")
        break


<: 07 06 01 00 04 40 1f 00 00 ec 69
